# Question 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape,  Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Adamax
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Embedding,Dense,Reshape,Flatten,Dropout


In [ ]:
# function for loading the data
def load_data(df_data):
  max_user_id = df_data['User id'].drop_duplicates().max()
  max_item_id = df_data['Item id'].drop_duplicates().max()
  df_data['user_emd_id'] = df_data['User id'] - 1
  df_data['item_emd_id'] = df_data['Item id'] - 1
  # print(str(len(df_data))+' ratings loaded')
  return max_user_id, max_item_id

load the data

In [ ]:
ratings = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u.data', 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['User id', 'Item id', 'Rating', 'Timestamp'])
df_ratings_train = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u1.base', delimiter='\t', names=['User id', 'Item id', 'Rating', 'Timestamp'])
df_ratings_test = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u1.test', delimiter='\t', names=['User id', 'Item id', 'Rating', 'Timestamp'])
# load the data
max_userid, max_movieid = load_data(ratings)
# load the train
num_of_users_train, num_of_items_train = load_data(df_ratings_train)
# load the test
num_of_users_test, num_of_items_test = load_data(df_ratings_test)

Define neural collaborative filtering model

In [ ]:
def get_ncf_model(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    print("ncf model")
    model.summary()

    return model

Prepare inputs to model

In [ ]:
Users = df_ratings_train['user_emd_id'].values
Items = df_ratings_train['item_emd_id'].values
Ratings = df_ratings_train['Rating'].values

## Train 1 

Compile model

In [ ]:
K_LATENT = 20
hidden_dim = 1
do = 0.5

NCF_Model = get_ncf_model(max_userid, max_movieid, K_LATENT, hidden_dim, do)
NCF_Model.compile(loss='mse', optimizer=Adamax(), metrics=['mae'])

ncf model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
____________________________________________________________________________________

Train model

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=5), 
             ModelCheckpoint('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights', save_best_only=True)]
history = NCF_Model.fit([Users, Items], Ratings, epochs=100, validation_split=.1, verbose=1, callbacks=callbacks, batch_size = 32)

Epoch 1/100
2250/2250 [==============================] - 4s 1ms/step - loss: 8.9320 - mae: 2.6709 - val_loss: 2.1976 - val_mae: 1.2875
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights/assets
Epoch 2/100
2250/2250 [==============================] - 3s 1ms/step - loss: 2.4688 - mae: 1.2753 - val_loss: 1.2552 - val_mae: 0.9243
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights/assets
Epoch 3/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.3349 - mae: 0.9364 - val_loss: 1.0854 - val_mae: 0.8762
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights/assets
Epoch 4/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.1740 - mae: 0.8885 - val_loss: 1.0804 - val_mae: 0.8728
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb

calculate the MAE

In [ ]:
def calculate_mae():
  test_users = df_ratings_test['user_emd_id'].values
  test_items = df_ratings_test['item_emd_id'].values
  test_rating = df_ratings_test['Rating'].values
  prediction = NCF_Model.predict([test_users,test_items])
  df_predict = pd.DataFrame(data=prediction, columns=['prediction'])
  df_predict['rating'] = test_rating
  df_predict['diffrence'] = abs(df_predict['rating']- df_predict['prediction'])

  print(df_predict)

  print('MAE:' + str(sum(df_predict['diffrence'])/len(df_predict['diffrence'])))

calculate_mae()

       prediction  rating  diffrence
0        3.595116       5   1.404884
1        3.661030       3   0.661030
2        3.963953       5   1.036047
3        3.660741       5   1.339259
4        3.469081       3   0.469081
...           ...     ...        ...
19995    3.677051       4   0.322949
19996    3.573693       4   0.426307
19997    3.346525       3   0.346525
19998    3.431136       3   0.431136
19999    3.428607       5   1.571393

[20000 rows x 3 columns]
MAE:0.8811644292831421


## train 2 - more hidden layers - 10 

compile model

In [ ]:
K_LATENT = 20
hidden_dim = 10
do = 0.5

NCF_Model = get_ncf_model(max_userid, max_movieid, K_LATENT, hidden_dim, do)
NCF_Model.compile(loss='mse', optimizer=Adamax(), metrics=['mae'])

ncf model
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________

train model

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=5), 
             ModelCheckpoint('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_2', save_best_only=True)]
history = NCF_Model.fit([Users, Items], Ratings, epochs=100, validation_split=.1, verbose=1, callbacks=callbacks, batch_size = 32)

Epoch 1/100
2250/2250 [==============================] - 4s 1ms/step - loss: 7.7280 - mae: 2.4203 - val_loss: 1.4190 - val_mae: 0.9900
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_2/assets
Epoch 2/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.8604 - mae: 1.1052 - val_loss: 1.2273 - val_mae: 0.9164
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_2/assets
Epoch 3/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.3582 - mae: 0.9440 - val_loss: 1.1150 - val_mae: 0.8712
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_2/assets
Epoch 4/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.1278 - mae: 0.8549 - val_loss: 1.0627 - val_mae: 0.8449
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k

calculate MAE

In [ ]:
calculate_mae()

       prediction  rating  diffrence
0        3.753603       5   1.246397
1        4.050681       3   1.050681
2        4.472354       5   0.527646
3        4.061982       5   0.938018
4        3.232677       3   0.232677
...           ...     ...        ...
19995    3.950520       4   0.049480
19996    3.720449       4   0.279551
19997    3.205225       3   0.205225
19998    3.603199       3   0.603199
19999    3.535588       5   1.464412

[20000 rows x 3 columns]
MAE:0.7534351791203022


## train 3 - using another optimizer - SGD

compile model

In [ ]:
K_LATENT = 20
hidden_dim = 20
do = 0.5

NCF_Model = get_ncf_model(max_userid, max_movieid, K_LATENT, hidden_dim, do)
NCF_Model.compile(loss='mse', optimizer=SGD(), metrics=['mae'])

ncf model
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________

train model

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=5), 
             ModelCheckpoint('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_3', save_best_only=True)]
history = NCF_Model.fit([Users, Items], Ratings, epochs=100, validation_split=.1, verbose=1, callbacks=callbacks, batch_size = 32)

Epoch 1/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.8202 - mae: 1.0697 - val_loss: 1.1027 - val_mae: 0.8771
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_3/assets
Epoch 2/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.1300 - mae: 0.8724 - val_loss: 1.0220 - val_mae: 0.8192
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_3/assets
Epoch 3/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.0017 - mae: 0.8007 - val_loss: 1.0075 - val_mae: 0.8064
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_3/assets
Epoch 4/100
2250/2250 [==============================] - 3s 1ms/step - loss: 0.9561 - mae: 0.7776 - val_loss: 0.9994 - val_mae: 0.8010
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k

calculate MAE

In [ ]:
calculate_mae()

       prediction  rating  diffrence
0        3.622347       5   1.377653
1        3.979993       3   0.979993
2        4.448256       5   0.551744
3        3.978531       5   1.021469
4        3.224342       3   0.224342
...           ...     ...        ...
19995    3.995329       4   0.004671
19996    3.716760       4   0.283240
19997    3.274203       3   0.274203
19998    3.547563       3   0.547563
19999    3.439081       5   1.560919

[20000 rows x 3 columns]
MAE:0.7577954462587834


## train 4 - larger size of layer - 30 nodes

compile model

In [ ]:
K_LATENT = 30
hidden_dim = 20
do = 0.5

NCF_Model = get_ncf_model(max_userid, max_movieid, K_LATENT, hidden_dim, do)
NCF_Model.compile(loss='mse', optimizer=Adamax(), metrics=['mae'])

ncf model
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 30)        28290       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 30)        50460       item_input[0][0]                 
__________________________________________________________________________________

train model

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=5), 
             ModelCheckpoint('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_4', save_best_only=True)]
history = NCF_Model.fit([Users, Items], Ratings, epochs=100, validation_split=.1, verbose=1, callbacks=callbacks, batch_size = 32)

Epoch 1/100
2250/2250 [==============================] - 4s 1ms/step - loss: 7.4672 - mae: 2.3617 - val_loss: 1.4389 - val_mae: 1.0015
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_4/assets
Epoch 2/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.8746 - mae: 1.1081 - val_loss: 1.1948 - val_mae: 0.9052
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_4/assets
Epoch 3/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.3653 - mae: 0.9470 - val_loss: 1.0793 - val_mae: 0.8557
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u_emb_weights_4/assets
Epoch 4/100
2250/2250 [==============================] - 3s 1ms/step - loss: 1.0975 - mae: 0.8444 - val_loss: 1.0274 - val_mae: 0.8298
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k

calculate MAE

In [ ]:
calculate_mae()

       prediction  rating  diffrence
0        3.576867       5   1.423133
1        3.978754       3   0.978754
2        4.421106       5   0.578894
3        4.025956       5   0.974044
4        3.088450       3   0.088450
...           ...     ...        ...
19995    4.050476       4   0.050476
19996    3.759901       4   0.240099
19997    2.986450       3   0.013550
19998    3.635630       3   0.635630
19999    3.544216       5   1.455784

[20000 rows x 3 columns]
MAE:0.7506155411958695


# Question 5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deepctr
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from deepctr.models import *
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [ ]:
# read the data
df_train_ratings = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u1.base', 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['User_id', 'Item_id', 'rating', 'timestamp'])
df_test_ratings = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u1.test', 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['User_id', 'Item_id', 'rating', 'timestamp'])
df_users_info = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u.user',delimiter='|',names=['User_id','age','gender','occupation','zip_code'])
df_items_info = pd.read_csv('/content/drive/Shareddrives/E-Commerce Project/ml-100k/ml-100k/u.item',delimiter='|', encoding = "ISO-8859-1", names=['Item_id', 'movie_title', 'release_date', 'video_release_date','IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation','Children', 'Comedy', 'Crime', 'Documentary', 'Drama','Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western'])

# merge the data for more features
df_temp_merge = pd.merge(df_train_ratings, df_users_info)
df_train_merged = pd.merge(df_temp_merge, df_items_info) 

df_temp_merge = pd.merge(df_test_ratings, df_users_info)
df_test_merged = pd.merge(df_temp_merge, df_items_info) 

In [ ]:
# define the relevant features for the model
sparse_features = ['User_id', 'Item_id', 'timestamp', 'gender']
target = ['rating']

In [ ]:
 # 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    df_train_merged[feat] = lbe.fit_transform(df_train_merged[feat])
    df_test_merged[feat] = lbe.fit_transform(df_test_merged[feat])

In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=df_train_merged[feat].max() + 1,embedding_dim=4)
                        for i,feat in enumerate(sparse_features)]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
# 3.generate input data for model

# train, test = train_test_split(df_merged, test_size=0.2, random_state=2020)
train = df_train_merged
test = df_test_merged
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}


In [ ]:
# 4.Define Model 1 ,train,predict and evaluate 
model_1 = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model_1.compile("adam", "mse",
              metrics=['mse'], )

history = model_1.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=30, verbose=2, validation_split=0.2, )
pred_ans = model_1.predict(test_model_input, batch_size=256)
print("mae", round(mean_absolute_error(test[target].values, pred_ans), 4))

Epoch 1/30
250/250 - 3s - loss: 2.4152 - mse: 2.4150 - val_loss: 1.1231 - val_mse: 1.1228
Epoch 2/30
250/250 - 1s - loss: 0.6742 - mse: 0.6736 - val_loss: 1.1557 - val_mse: 1.1549
Epoch 3/30
250/250 - 1s - loss: 0.4934 - mse: 0.4924 - val_loss: 1.1982 - val_mse: 1.1971
Epoch 4/30
250/250 - 1s - loss: 0.4316 - mse: 0.4304 - val_loss: 1.2371 - val_mse: 1.2358
Epoch 5/30
250/250 - 1s - loss: 0.3987 - mse: 0.3973 - val_loss: 1.2706 - val_mse: 1.2691
Epoch 6/30
250/250 - 1s - loss: 0.3784 - mse: 0.3767 - val_loss: 1.2880 - val_mse: 1.2862
Epoch 7/30
250/250 - 1s - loss: 0.3642 - mse: 0.3623 - val_loss: 1.2661 - val_mse: 1.2641
Epoch 8/30
250/250 - 1s - loss: 0.3527 - mse: 0.3506 - val_loss: 1.3096 - val_mse: 1.3074
Epoch 9/30
250/250 - 1s - loss: 0.3370 - mse: 0.3347 - val_loss: 1.3120 - val_mse: 1.3095
Epoch 10/30
250/250 - 1s - loss: 0.3167 - mse: 0.3141 - val_loss: 1.2942 - val_mse: 1.2915
Epoch 11/30
250/250 - 1s - loss: 0.2957 - mse: 0.2929 - val_loss: 1.2998 - val_mse: 1.2969
Epoch 12

model 2

In [ ]:
# Define Model 2 ,train,predict and evaluate 
model_2 = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model_2.compile("sgd", "mse",
              metrics=['mse'], )

history = model_2.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=30, verbose=2, validation_split=0.2, )
pred_ans = model_2.predict(test_model_input, batch_size=256)
print("mae", round(mean_absolute_error(test[target].values, pred_ans), 4))

Epoch 1/30
250/250 - 2s - loss: 1.5471 - mse: 1.5471 - val_loss: 1.5331 - val_mse: 1.5331
Epoch 2/30
250/250 - 1s - loss: 1.1697 - mse: 1.1697 - val_loss: 1.5361 - val_mse: 1.5361
Epoch 3/30
250/250 - 1s - loss: 1.1574 - mse: 1.1574 - val_loss: 1.5470 - val_mse: 1.5470
Epoch 4/30
250/250 - 1s - loss: 1.1431 - mse: 1.1431 - val_loss: 1.4660 - val_mse: 1.4660
Epoch 5/30
250/250 - 1s - loss: 1.1249 - mse: 1.1249 - val_loss: 1.4714 - val_mse: 1.4714
Epoch 6/30
250/250 - 1s - loss: 1.1001 - mse: 1.1001 - val_loss: 1.4109 - val_mse: 1.4109
Epoch 7/30
250/250 - 1s - loss: 1.0671 - mse: 1.0671 - val_loss: 1.4016 - val_mse: 1.4016
Epoch 8/30
250/250 - 1s - loss: 1.0239 - mse: 1.0239 - val_loss: 1.3551 - val_mse: 1.3551
Epoch 9/30
250/250 - 1s - loss: 0.9743 - mse: 0.9743 - val_loss: 1.3171 - val_mse: 1.3170
Epoch 10/30
250/250 - 1s - loss: 0.9295 - mse: 0.9295 - val_loss: 1.2680 - val_mse: 1.2679
Epoch 11/30
250/250 - 1s - loss: 0.8979 - mse: 0.8979 - val_loss: 1.2475 - val_mse: 1.2474
Epoch 12

change the features

In [ ]:
# define the relevant features for the model
sparse_features = ['User_id', 'Item_id', 'age', 'occupation']
target = ['rating']
 # 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    df_train_merged[feat] = lbe.fit_transform(df_train_merged[feat])
    df_test_merged[feat] = lbe.fit_transform(df_test_merged[feat])
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=df_train_merged[feat].max() + 1,embedding_dim=4)
                        for i,feat in enumerate(sparse_features)]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
# 3.generate input data for model

# train, test = train_test_split(df_merged, test_size=0.2, random_state=2020)
train = df_train_merged
test = df_test_merged
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

model 1

In [ ]:
# 4.Define Model 1,train,predict and evaluate
model_1 = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model_1.compile("adam", "mse",
              metrics=['mse'], )

history = model_1.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=30, verbose=2, validation_split=0.2, )
pred_ans = model_1.predict(test_model_input, batch_size=256)
print("mae", round(mean_absolute_error(test[target].values, pred_ans), 4))

Epoch 1/30
250/250 - 2s - loss: 2.3226 - mse: 2.3226 - val_loss: 1.1703 - val_mse: 1.1702
Epoch 2/30
250/250 - 1s - loss: 0.8644 - mse: 0.8643 - val_loss: 1.1721 - val_mse: 1.1720
Epoch 3/30
250/250 - 1s - loss: 0.8559 - mse: 0.8558 - val_loss: 1.1697 - val_mse: 1.1696
Epoch 4/30
250/250 - 1s - loss: 0.8492 - mse: 0.8491 - val_loss: 1.1699 - val_mse: 1.1698
Epoch 5/30
250/250 - 1s - loss: 0.8437 - mse: 0.8436 - val_loss: 1.1732 - val_mse: 1.1731
Epoch 6/30
250/250 - 1s - loss: 0.8315 - mse: 0.8313 - val_loss: 1.1823 - val_mse: 1.1821
Epoch 7/30
250/250 - 1s - loss: 0.8221 - mse: 0.8219 - val_loss: 1.1965 - val_mse: 1.1963
Epoch 8/30
250/250 - 1s - loss: 0.8150 - mse: 0.8148 - val_loss: 1.2056 - val_mse: 1.2053
Epoch 9/30
250/250 - 1s - loss: 0.8119 - mse: 0.8117 - val_loss: 1.2130 - val_mse: 1.2128
Epoch 10/30
250/250 - 1s - loss: 0.8025 - mse: 0.8023 - val_loss: 1.2196 - val_mse: 1.2193
Epoch 11/30
250/250 - 1s - loss: 0.7961 - mse: 0.7958 - val_loss: 1.2302 - val_mse: 1.2299
Epoch 12

model 2

In [ ]:
# Define Model 2,train,predict and evaluate
model_2 = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model_2.compile("sgd", "mse",
              metrics=['mse'], )

history = model_2.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=30, verbose=2, validation_split=0.2, )
pred_ans = model_2.predict(test_model_input, batch_size=256)
print("mae", round(mean_absolute_error(test[target].values, pred_ans), 4))

Epoch 1/30
250/250 - 2s - loss: 1.5743 - mse: 1.5743 - val_loss: 1.5313 - val_mse: 1.5313
Epoch 2/30
250/250 - 1s - loss: 1.1635 - mse: 1.1635 - val_loss: 1.5244 - val_mse: 1.5244
Epoch 3/30
250/250 - 1s - loss: 1.1489 - mse: 1.1489 - val_loss: 1.4836 - val_mse: 1.4836
Epoch 4/30
250/250 - 1s - loss: 1.1334 - mse: 1.1334 - val_loss: 1.4797 - val_mse: 1.4797
Epoch 5/30
250/250 - 1s - loss: 1.1160 - mse: 1.1160 - val_loss: 1.4702 - val_mse: 1.4702
Epoch 6/30
250/250 - 1s - loss: 1.0942 - mse: 1.0942 - val_loss: 1.4433 - val_mse: 1.4433
Epoch 7/30
250/250 - 1s - loss: 1.0642 - mse: 1.0642 - val_loss: 1.3958 - val_mse: 1.3958
Epoch 8/30
250/250 - 1s - loss: 1.0225 - mse: 1.0225 - val_loss: 1.3238 - val_mse: 1.3237
Epoch 9/30
250/250 - 1s - loss: 0.9726 - mse: 0.9725 - val_loss: 1.3222 - val_mse: 1.3222
Epoch 10/30
250/250 - 1s - loss: 0.9277 - mse: 0.9277 - val_loss: 1.2760 - val_mse: 1.2760
Epoch 11/30
250/250 - 1s - loss: 0.8970 - mse: 0.8970 - val_loss: 1.2373 - val_mse: 1.2373
Epoch 12